In [104]:
import bosdyn.client
import bosdyn.client.util
import bosdyn.client.robot_command as bdcrc
from bosdyn.client.robot_command import RobotCommandClient
from bosdyn.geometry import EulerZXY
from bosdyn.client.robot_command import RobotCommandBuilder
from bosdyn.client.image import ImageClient
from bosdyn.util import seconds_to_duration

from bosdyn.client.world_object import WorldObjectClient
from bosdyn.client.frame_helpers import get_a_tform_b, get_vision_tform_body, BODY_FRAME_NAME, VISION_FRAME_NAME

from bosdyn.api import (arm_command_pb2, geometry_pb2, robot_command_pb2, synchronized_command_pb2,
                        trajectory_pb2, world_object_pb2)

import cv2
from groundlight import Groundlight
import pdb
import time
import numpy as np
import matplotlib.pyplot as plt

from copy import deepcopy

from PIL import Image
import io
np.random.seed(0)

### Setup

In [2]:
#Setup
sdk = bosdyn.client.create_standard_sdk('ASTRO')
robot = sdk.create_robot('192.168.50.3')
id_client = robot.ensure_client('robot-id')

#Authenticate

#works in python but not in notebook
#bosdyn.client.util.authenticate(robot)

#works everywhere
robot.authenticate('user', 'r4kr99yw3hje')

#State
robot_state_client = robot.ensure_client('robot-state')

#Create command client
command_client = robot.ensure_client(RobotCommandClient.default_service_name)

#image client
image_client = robot.ensure_client(ImageClient.default_service_name)

In [105]:
world_object_client = robot.ensure_client(WorldObjectClient.default_service_name)

### Battery

In [3]:
#Battery

def getBattInfo():
    
    robot_state = robot_state_client.get_robot_state()
    
    battery_state = robot_state.battery_states[0]
    
    chargePercent = battery_state.charge_percentage
    
    runtimeRemaining = battery_state.estimated_runtime
    
    return chargePercent.value, runtimeRemaining.seconds

In [96]:
chargePercent, runtimeRemaining = getBattInfo()

print("Battery at:", chargePercent, "%. Robot has", runtimeRemaining, "seconds left")

Battery at: 86.0 %. Robot has 4869 seconds left


### Lease

In [5]:
lease_client = robot.ensure_client('lease')
lease = lease_client.acquire()
lease_keep_alive = bosdyn.client.lease.LeaseKeepAlive(lease_client)

In [65]:
#Power on and time_sync

robot.power_on(timeout_sec=20)

print(robot.is_powered_on())

robot.time_sync.wait_for_sync()

True


### Command

In [81]:
bdcrc.blocking_sit(command_client, timeout_sec = 10)

#### Commands:

In [ ]:
#Works
footprint_R_body = EulerZXY(yaw=0, roll=0, pitch=0)
cmd = RobotCommandBuilder.synchro_stand_command(footprint_R_body=footprint_R_body, body_height = 0)

In [111]:
#Works
cmd = RobotCommandBuilder.synchro_sit_command()

In [15]:
#Works
cmd = RobotCommandBuilder.synchro_stand_command()

In [7]:
#Works
cmd = RobotCommandBuilder.selfright_command()

In [23]:
cmd = RobotCommandBuilder.arm_pose_command(0.75,0,0,1,0,0,0,'flat_body')

In [114]:
cmd = RobotCommandBuilder.arm_stow_command()

In [13]:
#Works
cmd = RobotCommandBuilder.claw_gripper_open_fraction_command(0.0)

In [25]:
#Works
cmd = RobotCommandBuilder.claw_gripper_open_fraction_command(1.0)

In [ ]:
#Works but requires end_time_secs = time.time() + x seconds
cmd = RobotCommandBuilder.synchro_velocity_command(0, 0, 0.2)

In [61]:
#Also requires end_time_secs info
frameTreeSnapshot = robot.get_frame_tree_snapshot()
cmd = RobotCommandBuilder.synchro_trajectory_command_in_body_frame(0, 0, -np.pi/6, frame_tree_snapshot = frameTreeSnapshot)

In [112]:
cmd_id = command_client.robot_command(cmd, end_time_secs = time.time()+5)

### Arm Experimentation

In [39]:
position = geometry_pb2.Vec3(x=, y=y, z=z)
rotation = geometry_pb2.Quaternion(w=qw, x=qx, y=qy, z=qz)
hand_pose = geometry_pb2.SE3Pose(position=position, rotation=rotation)

duration = seconds_to_duration(seconds)
hand_pose_traj_point = trajectory_pb2.SE3TrajectoryPoint(pose=hand_pose,
                                                         time_since_reference=duration)
hand_trajectory = trajectory_pb2.SE3Trajectory(points=[hand_pose_traj_point])

arm_cartesian_command = arm_command_pb2.ArmCartesianCommand.Request(
    root_frame_name=frame_name, pose_trajectory_in_task=hand_trajectory)
arm_command = arm_command_pb2.ArmCommand.Request(
    arm_cartesian_command=arm_cartesian_command)
synchronized_command = synchronized_command_pb2.SynchronizedCommand.Request(
    arm_command=arm_command)
robot_command = robot_command_pb2.RobotCommand(synchronized_command=synchronized_command)

SyntaxError: invalid syntax (1711796558.py, line 1)

In [185]:
cmd = RobotCommandBuilder.follow_arm_command()

In [186]:
cmd_id = command_client.robot_command(cmd, end_time_secs = time.time()+5)

0.8195254015709299 0.17215149309793554 0.44248703846447957


In [164]:
xPosRand = np.random.uniform(0.6,0.9)
yPosRand = np.random.uniform(-0.4,0.4)
zPosRand = np.random.uniform(-0.1,0.8)

print(xPosRand, yPosRand, zPosRand)

arm_pos_list = [xPosRand,yPosRand,zPosRand]
dur_sec = 3

duration = seconds_to_duration(dur_sec)


arm_pos = geometry_pb2.Vec3(x=arm_pos_list[0], 
                            y=arm_pos_list[1], 
                            z=arm_pos_list[2])
hand_pose = geometry_pb2.SE3Pose(position=arm_pos)
hand_pose_traj_point = trajectory_pb2.SE3TrajectoryPoint(pose=hand_pose,
                                                         time_since_reference=duration)
hand_traj = trajectory_pb2.SE3Trajectory(points=[hand_pose_traj_point])



gaze_pos_list = [1.5,0,-0.5]
gaze_pos = geometry_pb2.Vec3(x=gaze_pos_list[0],
                             y=gaze_pos_list[1],
                             z=gaze_pos_list[2])
gaze_pos_traj_point = trajectory_pb2.Vec3TrajectoryPoint(point=gaze_pos)
gaze_traj = trajectory_pb2.Vec3Trajectory(points=[gaze_pos_traj_point])

gaze_cmd = arm_command_pb2.GazeCommand.Request(target_trajectory_in_frame1=gaze_traj,
                                               frame1_name='body',
                                               tool_trajectory_in_frame2=hand_traj,
                                               frame2_name='body')

arm_command = arm_command_pb2.ArmCommand.Request(arm_gaze_command=gaze_cmd)
synchronized_command = synchronized_command_pb2.SynchronizedCommand.Request(arm_command=arm_command)
command = robot_command_pb2.RobotCommand(synchronized_command=synchronized_command)
gaze_command_id = command_client.robot_command(command)
bdcrc.block_until_arm_arrives(command_client, gaze_command_id, timeout_sec = time.time() + 3.0)

0.6626630268284504 -0.270952385692003 0.48779749291885866


False

In [292]:
cmd_id = command_client.robot_command(cmd, end_time_secs = time.time()+5)

In [44]:
cmd = RobotCommandBuilder.arm_pose_command(-0.5,0,0.5,1,0,0,0,'flat_body')

In [27]:
cmd = RobotCommandBuilder.arm_gaze_command(3,0,0.5,'body')

In [105]:
cmd = RobotCommandBuilder.arm_gaze_command(-0.5,0,0.5,'body')

In [106]:
cmd_id = command_client.robot_command(cmd, end_time_secs = time.time()+3)

In [58]:
from google.protobuf import duration_pb2
from bosdyn.api import (arm_command_pb2, geometry_pb2, robot_command_pb2, synchronized_command_pb2,
                        trajectory_pb2)

from bosdyn.client.frame_helpers import GRAV_ALIGNED_BODY_FRAME_NAME, ODOM_FRAME_NAME, get_a_tform_b

from bosdyn.client import math_helpers

from bosdyn.client.robot_command import block_until_arm_arrives


NameError: name 'odom_T_flat_body' is not defined

In [158]:
cmd = RobotCommandBuilder.create_arm_joint_trajectory_point(0,0,0,0,0,0)

In [114]:
cmd = RobotCommandBuilder.follow_arm_command()

In [ ]:
cmd = RobotCommandBuilder.arm_pose_command(0.75,0,0,1,0,0,0,'flat_body')

In [151]:
cmd = RobotCommandBuilder.arm_gaze_command(-1,0,-0.5,'body')

In [ ]:
cmd_id = command_client.robot_command(cmd, end_time_secs = time.time()+5)

In [102]:
# while(True):
#     #Works
#     cmd = RobotCommandBuilder.claw_gripper_open_fraction_command(0.0)
#     cmd_id = command_client.robot_command(cmd, end_time_secs = time.time()+5)

#     time.sleep(0.5)

#     #Works
#     cmd = RobotCommandBuilder.claw_gripper_open_fraction_command(1.0)
#     cmd_id = command_client.robot_command(cmd, end_time_secs = time.time()+5)

#     time.sleep(0.5)

In [34]:
def pushup():
    cmd = RobotCommandBuilder.synchro_stand_command(body_height = -0.5)
    cmd_id = command_client.robot_command(cmd, end_time_secs = time.time()+2)
    time.sleep(0.5)
    cmd = RobotCommandBuilder.synchro_stand_command(body_height = 0)
    cmd_id = command_client.robot_command(cmd, end_time_secs = time.time()+2)
    time.sleep(0.5)

In [32]:
def check_success(graph_nav_client, command_id=-1):
    """Use a navigation command id to get feedback from the robot and sit when command succeeds."""
    if command_id == -1:
        # No command, so we have no status to check.
        return False
    status = graph_nav_client.navigation_feedback(command_id)
    if status.status == graph_nav_pb2.NavigationFeedbackResponse.STATUS_REACHED_GOAL:
        # Successfully completed the navigation commands!
        return True
    elif status.status == graph_nav_pb2.NavigationFeedbackResponse.STATUS_LOST:
        print("Robot got lost when navigating the route, the robot will now sit down.")
        return True
    elif status.status == graph_nav_pb2.NavigationFeedbackResponse.STATUS_STUCK:
        print("Robot got stuck when navigating the route, the robot will now sit down.")
        return True
    elif status.status == graph_nav_pb2.NavigationFeedbackResponse.STATUS_ROBOT_IMPAIRED:
        print("Robot is impaired.")
        return True
    else:
        # Navigation command is not complete yet.
        return False

In [33]:
#Image capture from hand cam and save, WORKS
def capAndShowImage(image_client, saveName):
    
    image_response = image_client.get_image_from_sources(["hand_color_image"])[0]
    image = Image.open(io.BytesIO(image_response.shot.image.data))
    image.save(saveName)
#     matImg = cv2.cvtColor(matImg, cv2.COLOR_BGR2RGB)
#     plt.figure(figsize = (16,8))
#     plt.imshow(matImg)
#     plt.show()
#    return image_response

In [18]:
pushup()

### Image Classification tasks

# GL API + MOTION DEMO

In [31]:
GROUNDLIGHT_API_TOKEN = 'api_2AaZ8QiCOvUFeHLJWGg8NTCPOAJ_uBTv8nu8U82Nv59ZHhi134HkAAirX4dwZw'

#gl = Groundlight(endpoint="https://device.integ.positronix.ai/device-api")

gl = Groundlight(api_token=GROUNDLIGHT_API_TOKEN)

# Call an API method (e.g., retrieve a list of detectors)
detectors = gl.list_detectors()

det = detectors.results[0]

In [32]:
timeoutSec = 5

In [51]:
capAndShowImage(image_client, 'handColorImage.jpg')

In [52]:
image_query = gl.submit_image_query(detector_id=det.id, image = "handColorImage.jpg")

In [ ]:
list(image_response.shot.DESCRIPTOR.fields_by_name)

In [71]:
image_response = image_client.get_image_from_sources(["hand_color_image"])[0]
print(image_response.shot.acquisition_time.seconds)

image_response = image_client.get_image_from_sources(["frontleft_fisheye_image"])[0]
print(image_response.shot.acquisition_time.seconds)

1655321632
1655321632


In [40]:
with open(upload_filepath + "/graph", "rb") as graph_file:
    # Load the graph from disk.
    data = graph_file.read()
    current_graph = map_pb2.Graph()
    current_graph.ParseFromString(data)

In [41]:
response = graph_nav_client.upload_graph(lease=lease.lease_proto,
                                               graph=current_graph,
                                               generate_new_anchoring=False)

In [165]:
list(response.DESCRIPTOR.fields_by_name)

['header', 'lease_use_result', 'status']

In [28]:
response = graph_nav_client.clear_graph()

NameError: name 'graph_nav_client' is not defined

In [53]:
for i in range(10):
    time.sleep(2)
    capAndShowImage(image_client, 'handColorImage.jpg')
    image_query = gl.submit_image_query(detector_id=det.id, image = "handColorImage.jpg")
    print('sent query through GL API', i)
    
    if image_query.result.label != 'FAIL':
        pushup()
#     # turn 180
#     frameTreeSnapshot = robot.get_frame_tree_snapshot()
#     cmd = RobotCommandBuilder.synchro_trajectory_command_in_body_frame(0, 0, 0.2, frame_tree_snapshot = frameTreeSnapshot)
#     cmd_id = cmd_id = command_client.robot_command(cmd, end_time_secs = time.time()+timeOutSec)
#     bdcrc.block_for_trajectory_cmd(command_client, cmd_id)

sent query through GL API 0
sent query through GL API 1
sent query through GL API 2
sent query through GL API 3
sent query through GL API 4
sent query through GL API 5
sent query through GL API 6
sent query through GL API 7
sent query through GL API 8
sent query through GL API 9


In [ ]:
cubePossesed = False

for loopNum in range(3):
    for wpInd in [10, 9]:
        nav_to_cmd_id = graph_nav_client.navigate_to(waypointDict[wpInd], timeoutSec)
        waitUntilFinishedNav(nav_to_cmd_id)
        if wpInd == 9:
            frameTreeSnapshot = robot.get_frame_tree_snapshot()
            cmd = RobotCommandBuilder.synchro_trajectory_command_in_body_frame(0, 0, np.pi/2, frame_tree_snapshot = frameTreeSnapshot)
            cmd_id = cmd_id = command_client.robot_command(cmd, end_time_secs = time.time()+timeoutSec)
            bdcrc.block_for_trajectory_cmd(command_client, cmd_id)
            res = checkForCubeAndGrasp(det, graph_nav_client)
            if(res):
                cubePossessed = True
            waypointDict = graphInit(graph_nav_client, upload_filepath)
        elif wpInd == 10:
            if cubePossesed:
                cmd = RobotCommandBuilder.claw_gripper_open_fraction_command(1.0)
                cmd_id = command_client.robot_command(cmd, end_time_secs = time.time()+timeoutSec)
                cubePossesed = False
                
        

In [125]:
timeoutSec = 5

#Works
cmd = RobotCommandBuilder.claw_gripper_open_fraction_command(1.0)
cmd_id = command_client.robot_command(cmd, end_time_secs = time.time()+timeoutSec)

for i in range(6):
    
    #forward two meters
    frameTreeSnapshot = robot.get_frame_tree_snapshot()
    cmd = RobotCommandBuilder.synchro_trajectory_command_in_body_frame(2, 0, 0, frame_tree_snapshot = frameTreeSnapshot)
    cmd_id = cmd_id = command_client.robot_command(cmd, end_time_secs = time.time()+timeoutSec)
    bdcrc.block_for_trajectory_cmd(command_client, cmd_id)
    
    # turn 180
    frameTreeSnapshot = robot.get_frame_tree_snapshot()
    cmd = RobotCommandBuilder.synchro_trajectory_command_in_body_frame(0, 0, np.pi, frame_tree_snapshot = frameTreeSnapshot)
    cmd_id = cmd_id = command_client.robot_command(cmd, end_time_secs = time.time()+timeoutSec)
    bdcrc.block_for_trajectory_cmd(command_client, cmd_id)

In [39]:
graph_nav_client.download_graph()

anchoring {
}

# GRAPH NAV, UPLOADING MAP STILL BREAKS ARM CAM

### GraphNav 

In [9]:
from bosdyn.client.graph_nav import GraphNavClient
from bosdyn.api.graph_nav import graph_nav_pb2
from bosdyn.api.graph_nav import map_pb2
from bosdyn.api.graph_nav import nav_pb2
from bosdyn.client.frame_helpers import get_odom_tform_body
from bosdyn.client.exceptions import ResponseError
from bosdyn.api import image_pb2

In [10]:
graph_nav_client = robot.ensure_client(GraphNavClient.default_service_name)
current_graph = None
current_waypoint_snapshots = dict()
current_edge_snapshots = dict()
upload_filepath = "/home/gluser/groundlight/bdspot/newGraph"

In [11]:
def upload_graph_and_snapshots(upload_filepath, current_graph, 
                               graph_nav_client, current_waypoint_snapshots, current_edge_snapshots):
    """Upload the graph and snapshots to the robot."""
    print("Loading the graph from disk into local storage...")
    with open(upload_filepath + "/graph", "rb") as graph_file:
        # Load the graph from disk.
        data = graph_file.read()
        current_graph = map_pb2.Graph()
        current_graph.ParseFromString(data)
        print("Loaded graph has {} waypoints and {} edges".format(
            len(current_graph.waypoints), len(current_graph.edges)))
    for waypoint in current_graph.waypoints:
        # Load the waypoint snapshots from disk.
        with open(upload_filepath + "/waypoint_snapshots/{}".format(waypoint.snapshot_id),
                  "rb") as snapshot_file:
            waypoint_snapshot = map_pb2.WaypointSnapshot()
            waypoint_snapshot.ParseFromString(snapshot_file.read())
            current_waypoint_snapshots[waypoint_snapshot.id] = waypoint_snapshot
    for edge in current_graph.edges:
        if len(edge.snapshot_id) == 0:
            continue
        # Load the edge snapshots from disk.
        with open(upload_filepath + "/edge_snapshots/{}".format(edge.snapshot_id),
                  "rb") as snapshot_file:
            edge_snapshot = map_pb2.EdgeSnapshot()
            edge_snapshot.ParseFromString(snapshot_file.read())
            current_edge_snapshots[edge_snapshot.id] = edge_snapshot
    # Upload the graph to the robot.
    print("Uploading the graph and snapshots to the robot...")
    true_if_empty = not len(current_graph.anchoring.anchors)
    response = graph_nav_client.upload_graph(lease=lease.lease_proto,
                                                   graph=current_graph,
                                                   generate_new_anchoring=true_if_empty)
#    response = graph_nav_client.upload_graph(graph=current_graph)
    
    # Upload the snapshots to the robot.
    
    for snapshot_id in response.unknown_waypoint_snapshot_ids:
        waypoint_snapshot = current_waypoint_snapshots[snapshot_id]
        graph_nav_client.upload_waypoint_snapshot(waypoint_snapshot)
        print("Uploaded {}".format(waypoint_snapshot.id))
    for snapshot_id in response.unknown_edge_snapshot_ids:
        edge_snapshot = current_edge_snapshots[snapshot_id]
        graph_nav_client.upload_edge_snapshot(edge_snapshot)
        print("Uploaded {}".format(edge_snapshot.id))

    # The upload is complete! Check that the robot is localized to the graph,
    # and if it is not, prompt the user to localize the robot before attempting
    # any navigation commands.
    localization_state = graph_nav_client.get_localization_state()
    if not localization_state.localization.waypoint_id:
        # The robot is not localized to the newly uploaded graph.
        print("\n")
        print("Upload complete! The robot is currently not localized to the map; please localize", \
               "the robot using commands (2) or (3) before attempting a navigation command.")
    return response

In [75]:
start_time = time.time()
response = upload_graph_and_snapshots(upload_filepath, current_graph, 
                           graph_nav_client, current_waypoint_snapshots, current_edge_snapshots)
end_time = time.time()

Loading the graph from disk into local storage...
Loaded graph has 27 waypoints and 29 edges
Uploading the graph and snapshots to the robot...


Upload complete! The robot is currently not localized to the map; please localize the robot using commands (2) or (3) before attempting a navigation command.


In [38]:
# response = graph_nav_client.clear_graph()

In [13]:
def set_initial_localization_fiducial(state_client, graph_nav_client):
    """Trigger localization when near a fiducial."""
    robot_state = state_client.get_robot_state()
    current_odom_tform_body = get_odom_tform_body(
        robot_state.kinematic_state.transforms_snapshot).to_proto()
    
    # Create an empty instance for initial localization since we are asking it to localize
    # based on the nearest fiducial.
    localization = nav_pb2.Localization()
    graph_nav_client.set_localization(initial_guess_localization=localization,
                                            ko_tform_body=current_odom_tform_body)

In [30]:
def waitUntilFinishedNav(nav_to_cmd_id):
    is_finished = False
    prevTimeoutTimerSec = time.time()
    while(not is_finished):
        curTime = time.time()
        if(curTime - prevTimeoutTimerSec > timeoutSec):
            break
        time.sleep(0.2)
        is_finished = check_success(graph_nav_client, nav_to_cmd_id)

In [68]:
def spinInitFiducial(command_client, graph_nav_client, robot_state_client):
    timeoutSec = 5

    bdcrc.blocking_stand(command_client, timeout_sec = timeoutSec)
    
    for i in range(12):
        print("attempt #: ", i)
        frameTreeSnapshot = robot.get_frame_tree_snapshot()
        cmd = RobotCommandBuilder.synchro_trajectory_command_in_body_frame(0, 0, np.pi/6, frame_tree_snapshot = frameTreeSnapshot)
        cmd_id = cmd_id = command_client.robot_command(cmd, end_time_secs = time.time()+timeoutSec)
        bdcrc.block_for_trajectory_cmd(command_client, cmd_id)
        try:
            set_initial_localization_fiducial(robot_state_client, graph_nav_client)
            break
        except ResponseError as e:
            print("Error during fiducial localization init: ", e)

In [63]:
#Also requires end_time_secs info
frameTreeSnapshot = robot.get_frame_tree_snapshot()
cmd = RobotCommandBuilder.synchro_trajectory_command_in_body_frame(0, 0, -np.pi/6, frame_tree_snapshot = frameTreeSnapshot)

In [64]:
cmd_id = cmd_id = command_client.robot_command(cmd, end_time_secs = time.time()+5)

In [60]:
cmd = RobotCommandBuilder.synchro_trajectory_command_in_body_frame(0, 0, np.pi/6, frame_tree_snapshot = frameTreeSnapshot)
cmd_id = cmd_id = command_client.robot_command(cmd, end_time_secs = time.time()+5)
#bdcrc.block_for_trajectory_cmd(command_client, cmd_id)

In [20]:
try:
    set_initial_localization_fiducial(robot_state_client, graph_nav_client)
except ResponseError as e:
    print("Error during fiducial localization init: ", e)

Error during fiducial localization init:  bosdyn.api.graph_nav.SetLocalizationResponse (ResponseError): Code: 8 (STATUS_NO_MATCHING_FIDUCIAL)


In [24]:
#Gets the current graph and reads off waypoints

current_graph = graph_nav_client.download_graph()

waypointList = [(waypoint.annotations.name,waypoint.id) for waypoint in current_graph.waypoints]

waypointDict = dict()

#make a dict and exclude default waypoints

for waypointName, waypointId in waypointList:
    
    if(waypointName == 'default'):
        continue
    
    waypointDict[int(waypointName[9:])] = waypointId

In [36]:
timeoutSec = 10

for wpInd in sorted(list(waypointDict.keys()))[0:1]:
    nav_to_cmd_id = graph_nav_client.navigate_to(waypointDict[wpInd], timeoutSec)
    print("moving to:", wpInd, "with cmd_id:", nav_to_cmd_id)
    is_finished = False
    prevTimeoutTimerSec = time.time()
    while(not is_finished):
        curTime = time.time()
        if(curTime - prevTimeoutTimerSec > timeoutSec):
            break
        time.sleep(0.2)
        is_finished = check_success(graph_nav_client, nav_to_cmd_id)
        
    print("finished moving to:", wpInd)
    

moving to: 0 with cmd_id: 24
finished moving to: 0


In [98]:
request_fiducials = [world_object_pb2.WORLD_OBJECT_APRILTAG]

In [107]:
fiducial_objects = world_object_client.list_world_objects(
            object_type=request_fiducials).world_objects

In [109]:
fiducial_objects[0]

id: 22
name: "world_obj_apriltag_583"
apriltag_properties {
  tag_id: 583
  dimensions {
    x: 0.1459999978542328
    y: 0.1459999978542328
  }
  frame_name_fiducial: "fiducial_583"
  frame_name_fiducial_filtered: "filtered_fiducial_583"
  detection_covariance {
    matrix {
      rows: 6
      cols: 6
      values: 0.0010554356595546513
      values: -0.0005524812856049933
      values: -0.00038865561918043995
      values: 0.000560126071521994
      values: -0.00010505305977088483
      values: 0.0008553801664207153
      values: -0.0005524812856049936
      values: 0.0005799497829589058
      values: -8.675788398989223e-05
      values: -0.00019142884554062975
      values: -0.0002610303638402707
      values: -9.476722068416637e-05
      values: -0.0003886556191804397
      values: -8.675788398989246e-05
      values: 0.0005138579058245023
      values: -0.000352456968209605
      values: 0.0004175498887450027
      values: -0.0006837422843684991
      values: 0.000560126071521994

In [100]:
request_fiducials[0]

2

In [70]:
nav_to_cmd_id = graph_nav_client.navigate_to(waypointDict[10], timeoutSec)

In [95]:
nav_to_cmd_id = graph_nav_client.navigate_to(waypointDict[9], timeoutSec)
waitUntilFinishedNav(nav_to_cmd_id)

cmd = RobotCommandBuilder.synchro_trajectory_command_in_body_frame(0, 0, np.pi/2, frame_tree_snapshot = frameTreeSnapshot)
cmd_id = cmd_id = command_client.robot_command(cmd, end_time_secs = time.time()+timeoutSec)
bdcrc.block_for_trajectory_cmd(command_client, cmd_id)

graph_nav_client.clear_graph()

In [76]:
spinInitFiducial(command_client, graph_nav_client, robot_state_client)

attempt #:  0
